In [ ]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

In [ ]:
import os
AA_TOKEN=os.getenv("AA_TOKEN")
NAMESPACE=os.getenv("AA_NAMESPACE")

# Question & Answer

## SingleChunkQA

In [ ]:
from typing import Sequence
from intelligence_layer.core import (
    Task,
    TextHighlight,
    CompleteInput,
    TaskSpan,
    LuminousControlModel,
    NoOpTracer,
    TextHighlightInput,
)
from pydantic import BaseModel


class Input(BaseModel):
    chunk: str
    question: str


class Output(BaseModel):
    answer: str
    highlights: Sequence[str]


class SingleChunkQaTask(Task[Input, Output]):
    def __init__(
        self,
        model: LuminousControlModel | None = None,
    ):
        super().__init__()
        self._model = model or LuminousControlModel("luminous-nextgen-7b-control-384k")

    def do_run(self, input: Input, task_span: TaskSpan) -> Output:
        instruction = f'Beantworte die Frage anhand des Textes. Wenn sich die Frage nicht mit dem Text beantworten lässt, antworte "Keine Antwort in den Quellen".\nQuellen: {input.chunk}\nFrage: {input.question}'
        prompt = self._model.to_instruct_prompt(instruction=instruction)
        input = CompleteInput(prompt=prompt)
        output = self._model.complete(input, task_span)

        highlight_task = TextHighlight(model=self._model)
        highlight_input = TextHighlightInput(
            rich_prompt=prompt,
            target=output.completion,
        )
        highlight_output = highlight_task.run(highlight_input, task_span)

        print(prompt.items)
        return Output(
            answer=output.completion,
            highlights=[
                prompt.items[0].text[highlight.start : highlight.end]
                for highlight in highlight_output.highlights
                if highlight.score > 0
            ],
        )


question = "Wer mag Pizza?"
chunk = "Tina mag nicht so gern Pizza. Mike aber mag Pizza sehr."

task = SingleChunkQaTask()
result = task.run(input=Input(chunk=chunk, question=question), tracer=NoOpTracer())
result

In [ ]:
chunk
result.highlights

## MultiChunkRetrieverQA

In [ ]:
collection_name = "rag-collection"
index_name = "rag-index"
number_of_chunks = 5
query = "Was ist nohtyp?"

In [ ]:
from intelligence_layer.connectors import (
    DocumentIndexClient,
    DocumentIndexRetriever,
)
from intelligence_layer.core import InMemoryTracer, LuminousControlModel
from intelligence_layer.examples import MultipleChunkRetrieverQa, RetrieverBasedQaInput


document_index = DocumentIndexClient(
    token=AA_TOKEN,
)

document_index_retriever = DocumentIndexRetriever(
    document_index=document_index,
    index_name=index_name,
    namespace=NAMESPACE,
    collection=collection_name,
    k=5,
    threshold=0.5,
)

model = LuminousControlModel("luminous-nextgen-7b-control-384k")
retriever_qa = MultipleChunkRetrieverQa(
    document_index_retriever, 
    insert_chunk_number=number_of_chunks,
)

input = RetrieverBasedQaInput(
    question=query,
)
tracer = InMemoryTracer()

output = retriever_qa.run(input, tracer)
print("Answer: ", output.answer)
print("-"*50)
print("Information:")
print(output.model_dump_json(indent=2))

In [ ]:
tracer